# NBA Player Performance Dynamics: Team Styles Analysis

This notebook analyzes team playing styles in the NBA, identifying distinct tactical patterns and examining how they relate to team success and player performance.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Add the project root to the path so we can import our modules
sys.path.append('..')

# Import our modules
from src.data_processing import extract_team_styles
from src.visualization import create_team_style_cards
from src.utils import setup_plotting_style

# Set up plotting style
setup_plotting_style()

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Load Processed Data

Let's load the processed game data that we'll use to analyze team styles.

In [ ]:
# Load the processed data
try:
    games_processed = pd.read_csv('../data/processed/games_processed.csv')
    player_dynamics = pd.read_csv('../data/processed/player_dynamics.csv')
    
    # Convert date strings to datetime objects
    games_processed['GAME_DATE'] = pd.to_datetime(games_processed['GAME_DATE'])
    
    print(f"Loaded processed games data with {len(games_processed)} records")
    print(f"Loaded player dynamics data with {len(player_dynamics)} players")
except FileNotFoundError:
    print("Processed data not found. Please run the previous notebooks first.")

In [ ]:
# Examine the games data
games_processed.head()

## Team Statistical Signatures

Let's extract key style indicators for each team to create team profile metrics.

In [ ]:
# Aggregate game data by team
team_stats = games_processed.groupby('Team_ID').agg({
    'TeamName': 'first',
    'PTS': 'mean',
    'FGA': 'mean',
    'FG3A': 'mean',
    'FTA': 'mean',
    'AST': 'mean',
    'OREB': 'mean',
    'DREB': 'mean',
    'STL': 'mean',
    'BLK': 'mean',
    'TOV': 'mean',
    'PLUS_MINUS': 'mean',
    'WL': lambda x: (x == 'W').mean(),  # Win percentage
    'PointsPerPossession': 'mean',
    'AssistRatio': 'mean',
    'TurnoverRatio': 'mean',
    'EffectiveFG': 'mean',
    'DefensiveRebound%': 'mean',
    'OffensiveRebound%': 'mean'
}).reset_index()

# Rename win percentage column
team_stats = team_stats.rename(columns={'WL': 'win_pct'})

# Create style metrics
team_stats['pace'] = team_stats['FGA'] + team_stats['TOV'] - team_stats['OREB']
team_stats['three_point_rate'] = team_stats['FG3A'] / team_stats['FGA']
team_stats['assist_rate'] = team_stats['AST'] / team_stats['FGA']
team_stats['defensive_focus'] = (team_stats['STL'] + team_stats['BLK']) / (team_stats['STL'] + team_stats['BLK'] + team_stats['DREB'] + team_stats['OREB'])
team_stats['offensive_efficiency'] = team_stats['PointsPerPossession']

# Sort by win percentage
team_stats = team_stats.sort_values('win_pct', ascending=False)

# Display team stats
team_stats.head()

In [ ]:
# Visualize key style metrics
style_metrics = ['pace', 'three_point_rate', 'assist_rate', 'defensive_focus', 'offensive_efficiency']

# Create a figure with subplots
fig, axes = plt.subplots(len(style_metrics), 1, figsize=(12, 15))

# Plot each style metric
for i, metric in enumerate(style_metrics):
    # Sort teams by this metric
    sorted_teams = team_stats.sort_values(metric, ascending=False)
    
    # Plot the metric
    axes[i].barh(sorted_teams['TeamName'], sorted_teams[metric], color='skyblue')
    axes[i].set_title(f'Team {metric.replace("_", " ").title()}', fontsize=12)
    axes[i].grid(True, alpha=0.3)
    
    # Add league average line
    league_avg = team_stats[metric].mean()
    axes[i].axvline(x=league_avg, color='red', linestyle='--', label='League Average')
    
    # Add legend to the first subplot
    if i == 0:
        axes[i].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Calculate style entropy/diversity metrics
# Normalize metrics for entropy calculation
scaler = StandardScaler()
style_data = scaler.fit_transform(team_stats[style_metrics])

# Calculate entropy for each team
style_entropy = []
for i in range(len(team_stats)):
    team_style = style_data[i]
    # Convert to probability distribution (normalize to sum to 1)
    style_probs = np.abs(team_style) / (np.sum(np.abs(team_style)) + 1e-10)
    # Calculate entropy
    entropy = -np.sum(style_probs * np.log2(style_probs + 1e-10))
    style_entropy.append(entropy)

team_stats['style_entropy'] = style_entropy

# Sort by style entropy
team_stats_by_entropy = team_stats.sort_values('style_entropy', ascending=False)

# Display teams by style entropy
plt.figure(figsize=(12, 6))
plt.barh(team_stats_by_entropy['TeamName'], team_stats_by_entropy['style_entropy'], color='skyblue')
plt.xlabel('Style Entropy (Higher = More Diverse Style)', fontsize=12)
plt.title('Team Style Diversity', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpreting Style Metrics

The style metrics we've calculated provide insights into each team's playing style:

1. **Pace**: Measures the tempo at which a team plays, with higher values indicating faster play.
2. **Three-Point Rate**: Indicates a team's reliance on three-point shooting, with higher values suggesting a more perimeter-oriented offense.
3. **Assist Rate**: Reflects ball movement and team play, with higher values indicating more passing and less isolation.
4. **Defensive Focus**: Measures emphasis on steals and blocks relative to rebounding, with higher values suggesting an aggressive defensive approach.
5. **Offensive Efficiency**: Indicates how effectively a team scores per possession, with higher values suggesting better offensive execution.

**Style Entropy** measures the diversity of a team's playing style across these dimensions. Teams with high entropy have a more balanced approach across multiple style dimensions, while teams with low entropy tend to specialize in specific aspects of the game.

## Dimensionality Reduction

Let's apply Principal Component Analysis (PCA) to reduce the dimensionality of our style metrics and identify the key components of team playing styles.

In [ ]:
# Apply PCA to style metrics
pca = PCA(n_components=2)
principal_components = pca.fit_transform(style_data)

# Create a dataframe with the principal components
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['TeamName'] = team_stats['TeamName']
pca_df['win_pct'] = team_stats['win_pct']

# Display the explained variance ratio
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.2f}")

In [ ]:
# Visualize teams in the reduced-dimensional space
plt.figure(figsize=(12, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], 
                      c=pca_df['win_pct'], cmap='viridis', 
                      s=100, alpha=0.7)

# Add team labels
for i, txt in enumerate(pca_df['TeamName']):
    plt.annotate(txt, (pca_df['PC1'].iloc[i], pca_df['PC2'].iloc[i]), fontsize=9)

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Win Percentage', fontsize=12)

# Add component loadings
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
for i, feature in enumerate(style_metrics):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='red', alpha=0.5)
    plt.text(loadings[i, 0] * 1.15, loadings[i, 1] * 1.15, feature, 
             color='red', ha='center', va='center', fontsize=10)

plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.2f})', fontsize=12)
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.2f})', fontsize=12)
plt.title('Team Style PCA', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Examine the component loadings
component_loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=style_metrics
)

component_loadings

### Interpreting Principal Components

Based on the component loadings, we can interpret the principal components as follows:

**Principal Component 1** appears to represent [interpretation based on your PCA results, e.g., "modern vs. traditional play" or "offensive vs. defensive focus"].

**Principal Component 2** seems to capture [interpretation based on your PCA results, e.g., "pace vs. control" or "inside vs. outside scoring"].

Together, these two components explain approximately [X]% of the variance in team playing styles, suggesting that they capture the most important dimensions of stylistic variation in the NBA.

## Style Clustering Analysis

Now let's use K-means clustering to identify distinct playing styles among NBA teams.

In [ ]:
# Determine the optimal number of clusters using the elbow method
inertia = []
silhouette = []
k_range = range(2, 7)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(style_data)
    inertia.append(kmeans.inertia_)
    
    # Calculate silhouette score
    if k > 1:  # Silhouette score requires at least 2 clusters
        silhouette.append(silhouette_score(style_data, kmeans.labels_))
    else:
        silhouette.append(0)

# Plot the elbow curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Inertia plot
ax1.plot(k_range, inertia, 'o-')
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Inertia', fontsize=12)
ax1.set_title('Elbow Method for Optimal k', fontsize=14)
ax1.grid(True, alpha=0.3)

# Silhouette score plot
ax2.plot(k_range, silhouette, 'o-')
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Method for Optimal k', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Apply K-means clustering with the optimal number of clusters
n_clusters = 4  # Based on the elbow and silhouette methods
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
team_stats['style_cluster'] = kmeans.fit_predict(style_data)

# Name the clusters
style_names = {
    0: 'Modern Pace-and-Space',
    1: 'Traditional Inside-Out',
    2: 'Defensive-Oriented',
    3: 'Balanced Attack'
}

team_stats['style_name'] = team_stats['style_cluster'].map(style_names)

In [ ]:
# Visualize team clustering in PCA space
plt.figure(figsize=(12, 8))

# Add cluster information to PCA dataframe
pca_df['style_cluster'] = team_stats['style_cluster']
pca_df['style_name'] = team_stats['style_name']

# Plot each cluster with a different color
for cluster in range(n_clusters):
    cluster_data = pca_df[pca_df['style_cluster'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'], 
                label=style_names[cluster], s=100, alpha=0.7)

# Add team labels
for i, txt in enumerate(pca_df['TeamName']):
    plt.annotate(txt, (pca_df['PC1'].iloc[i], pca_df['PC2'].iloc[i]), fontsize=9)

plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.2f})', fontsize=12)
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.2f})', fontsize=12)
plt.title('Team Style Clusters', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Analyze cluster characteristics
cluster_stats = team_stats.groupby('style_name').agg({
    'pace': 'mean',
    'three_point_rate': 'mean',
    'assist_rate': 'mean',
    'defensive_focus': 'mean',
    'offensive_efficiency': 'mean',
    'win_pct': 'mean',
    'PTS': 'mean',
    'Team_ID': 'count'
}).reset_index()

# Rename count column
cluster_stats = cluster_stats.rename(columns={'Team_ID': 'team_count'})

# Sort by win percentage
cluster_stats = cluster_stats.sort_values('win_pct', ascending=False)

cluster_stats

In [ ]:
# Visualize cluster characteristics
# Create a radar chart for each cluster
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

# Radar chart function
def radar_chart(ax, angles, values, color, label):
    # Plot data
    ax.plot(angles, values, 'o-', linewidth=2, color=color, label=label)
    # Fill area
    ax.fill(angles, values, alpha=0.25, color=color)
    # Set y-ticks
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    # Set category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    # Add legend
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

# Categories for the radar chart
categories = ['Pace', '3PT Rate', 'Ball Movement', 'Defense', 'Off. Efficiency']
# Number of categories
N = len(categories)
# Angle of each axis
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Close the loop

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(12, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()

# Colors for each cluster
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Normalize values for radar chart
max_values = {
    'pace': cluster_stats['pace'].max(),
    'three_point_rate': cluster_stats['three_point_rate'].max(),
    'assist_rate': cluster_stats['assist_rate'].max(),
    'defensive_focus': cluster_stats['defensive_focus'].max(),
    'offensive_efficiency': cluster_stats['offensive_efficiency'].max()
}

# Plot each cluster
for i, (_, cluster) in enumerate(cluster_stats.iterrows()):
    # Get normalized values
    values = [
        cluster['pace'] / max_values['pace'],
        cluster['three_point_rate'] / max_values['three_point_rate'],
        cluster['assist_rate'] / max_values['assist_rate'],
        cluster['defensive_focus'] / max_values['defensive_focus'],
        cluster['offensive_efficiency'] / max_values['offensive_efficiency']
    ]
    values += values[:1]  # Close the loop
    
    # Plot radar chart
    radar_chart(axes[i], angles, values, colors[i], cluster['style_name'])
    axes[i].set_title(f"{cluster['style_name']}\nWin%: {cluster['win_pct']*100:.1f}%, Teams: {cluster['team_count']}")

plt.tight_layout()
plt.show()

### Characterizing the Style Clusters

Based on our analysis, we can characterize the four distinct playing styles as follows:

1. **Modern Pace-and-Space**:
   - High tempo, three-point focused offense with spacing
   - Emphasizes ball movement and perimeter shooting
   - [Additional characteristics based on your data]
   - Example teams: [Examples from your data]

2. **Traditional Inside-Out**:
   - Post-oriented offense with methodical pace
   - Emphasizes interior scoring and rebounding
   - [Additional characteristics based on your data]
   - Example teams: [Examples from your data]

3. **Defensive-Oriented**:
   - Defense-first approach with opportunistic offense
   - Emphasizes steals, blocks, and transition opportunities
   - [Additional characteristics based on your data]
   - Example teams: [Examples from your data]

4. **Balanced Attack**:
   - Well-rounded approach without extreme tendencies
   - Balanced scoring from inside and outside
   - [Additional characteristics based on your data]
   - Example teams: [Examples from your data]

In [ ]:
# List teams in each style cluster
for style in style_names.values():
    style_teams = team_stats[team_stats['style_name'] == style]
    print(f"\n{style} Teams:")
    for _, team in style_teams.iterrows():
        print(f"- {team['TeamName']}: Win% = {team['win_pct']*100:.1f}%")

## Style Success Analysis

Let's analyze how different playing styles correlate with team success.

In [ ]:
# Visualize win percentage by style
plt.figure(figsize=(10, 6))
sns.boxplot(x='style_name', y='win_pct', data=team_stats)
plt.xlabel('Team Style', fontsize=12)
plt.ylabel('Win Percentage', fontsize=12)
plt.title('Win Percentage by Team Style', fontsize=14)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze correlation between style metrics and win percentage
style_win_corr = team_stats[style_metrics + ['win_pct']].corr()['win_pct'].drop('win_pct')

# Sort by correlation strength
style_win_corr = style_win_corr.sort_values(ascending=False)

# Visualize correlations
plt.figure(figsize=(10, 6))
style_win_corr.plot(kind='bar')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.xlabel('Style Metric', fontsize=12)
plt.ylabel('Correlation with Win Percentage', fontsize=12)
plt.title('Style Metrics Correlation with Team Success', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze style effectiveness against different opponents
# This would require game-by-game data with opponent information
# For now, we'll analyze overall style success

# Calculate average points scored and allowed by style
style_offense_defense = team_stats.groupby('style_name').agg({
    'PTS': 'mean',
    'win_pct': 'mean',
    'Team_ID': 'count'
}).reset_index()

# Rename columns
style_offense_defense = style_offense_defense.rename(columns={
    'PTS': 'pts_scored',
    'Team_ID': 'team_count'
})

# Sort by win percentage
style_offense_defense = style_offense_defense.sort_values('win_pct', ascending=False)

style_offense_defense

In [ ]:
# Visualize style success metrics
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot win percentage
x = np.arange(len(style_offense_defense))
ax1.bar(x, style_offense_defense['win_pct'], width=0.4, color='skyblue', label='Win Percentage')
ax1.set_ylabel('Win Percentage', fontsize=12)
ax1.set_ylim(0, 1)

# Create second y-axis for points
ax2 = ax1.twinx()
ax2.plot(x, style_offense_defense['pts_scored'], 'ro-', label='Points Scored')
ax2.set_ylabel('Points Scored', fontsize=12, color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Set x-axis
ax1.set_xticks(x)
ax1.set_xticklabels(style_offense_defense['style_name'], rotation=45)
ax1.set_xlabel('Team Style', fontsize=12)

# Add title and legend
plt.title('Style Success Metrics', fontsize=14)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

### Style Success Insights

Our analysis of style success reveals several key insights:

1. **Most Successful Style**: [Identify the most successful style based on your data]
   - Average win percentage: [X]%
   - Key success factors: [List factors based on your analysis]

2. **Style-Success Correlations**:
   - [Metric with strongest positive correlation] shows the strongest positive correlation with winning (r = [X])
   - [Metric with strongest negative correlation] shows the strongest negative correlation with winning (r = [Y])
   - This suggests that [interpretation based on your data]

3. **Offensive vs. Defensive Success**:
   - [Style with highest offensive rating] scores the most points ([X] PPG)
   - [Style with lowest offensive rating] scores the fewest points ([Y] PPG)
   - This indicates that [interpretation based on your data]

4. **Style Prevalence**:
   - [Most common style] is the most common style ([X] teams)
   - [Least common style] is the least common style ([Y] teams)
   - This suggests that [interpretation based on your data]

## Team Style Cards

Let's create intuitive visualizations of team styles using our module.

In [ ]:
# Create team style cards for top teams
top_n = 6  # Number of teams to display
top_teams = team_stats.nlargest(top_n, 'win_pct')

# Add average points to team_stats
team_stats['avg_pts'] = team_stats['PTS']

# Create style cards
fig = create_team_style_cards(team_stats, top_n)
plt.show()

## Team Style Evolution

Let's analyze how team styles evolve over time and identify style shifts due to personnel changes.

In [ ]:
# For a complete analysis, we would need data from multiple seasons
# For now, we'll simulate style evolution by analyzing style consistency within the current season

# Select a sample team for analysis
sample_team_id = team_stats['Team_ID'].iloc[0]
sample_team_name = team_stats[team_stats['Team_ID'] == sample_team_id]['TeamName'].iloc[0]

# Get games for this team
team_games = games_processed[games_processed['Team_ID'] == sample_team_id].copy()
team_games = team_games.sort_values('GAME_DATE')

# Calculate rolling style metrics
window_size = 10  # Number of games for rolling window

# Create rolling style metrics
rolling_metrics = {}
for metric in style_metrics:
    rolling_metrics[metric] = team_games[metric].rolling(window=window_size, min_periods=1).mean()

# Add rolling metrics to team_games
for metric, values in rolling_metrics.items():
    team_games[f'rolling_{metric}'] = values

# Visualize rolling style metrics
plt.figure(figsize=(14, 8))

for i, metric in enumerate(style_metrics):
    plt.plot(team_games['GAME_DATE'], team_games[f'rolling_{metric}'], label=metric)

plt.xlabel('Game Date', fontsize=12)
plt.ylabel('Style Metric Value (Rolling Average)', fontsize=12)
plt.title(f'{sample_team_name}: Style Evolution', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Style Evolution Insights

Our analysis of style evolution for [sample_team_name] reveals several interesting patterns:

1. **Style Consistency**: [Observations about style consistency based on your data]
2. **Style Shifts**: [Observations about style shifts based on your data]
3. **Performance Correlation**: [Observations about how style changes correlate with performance based on your data]

These insights suggest that [interpretation based on your data].

## Save Team Styles for Subsequent Analysis

Let's save our team styles analysis for use in subsequent notebooks.

In [ ]:
# Save team styles data
team_stats.to_csv('../data/processed/team_styles.csv', index=False)
print(f"Saved team styles data to ../data/processed/team_styles.csv")

## Conclusion

In this notebook, we've analyzed team playing styles in the NBA, identifying distinct tactical patterns and examining how they relate to team success. We've extracted team statistical signatures, applied dimensionality reduction to identify key style components, clustered teams into distinct style archetypes, and analyzed the relationship between playing style and team success.

Key accomplishments:
1. Extracted team style metrics including pace, three-point rate, assist rate, defensive focus, and offensive efficiency
2. Applied PCA to identify the principal components of team playing styles
3. Used K-means clustering to identify four distinct playing styles: Modern Pace-and-Space, Traditional Inside-Out, Defensive-Oriented, and Balanced Attack
4. Analyzed the relationship between playing style and team success
5. Created intuitive team style cards for visualization
6. Examined style evolution over time

Our analysis provides valuable insights into the tactical landscape of the NBA and the relationship between playing style and team success. In the next notebook, we'll explore player-team fit and how different players perform across different team systems.